In [ ]:
!pip install dlib
!pip install mtcnn
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
from google.colab import drive
from tqdm import tqdm
import numpy as np
import random
import os

#preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2

#face detection
from mtcnn import MTCNN
import dlib

#torch
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models, transforms

In [ ]:
drive.mount('/content/drive')

#put the name of your folder (or the relative path)
folder_name = "Dataset"
folder_path = f"/content/drive/My Drive/deepfake_det_task3/{folder_name}/"

os.listdir(folder_path)

In [ ]:
def get_image_paths(base_path):
    image_paths = []
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

In [ ]:
real_train_path = os.path.join(folder_path, "Train", "Real")
fake_train_path = os.path.join(folder_path, "Train", "Fake")
real_test_path = os.path.join(folder_path, "Test", "Real")
fake_test_path = os.path.join(folder_path, "Test", "Fake")

In [ ]:
#extract paths and labels for all images of training data
real_train_images = get_image_paths(real_train_path)
fake_train_images = get_image_paths(fake_train_path)
train_set = real_train_images + fake_train_images

real_test_images = get_image_paths(real_test_path)
fake_test_images = get_image_paths(fake_test_path)
test_set = real_test_images + fake_test_images

In [ ]:
train_labels = [1] * len(real_train_images) + [0] * len(fake_train_images)
test_labels = [1] * len(real_test_images) + [0] * len(fake_test_images)

In [ ]:
combined = list(zip(train_set, train_labels))  # Combina immagini e etichette
random.shuffle(combined)  # Mescola
train_set, train_labels = zip(*combined)  # Separa di nuovo

# Converti le tuple risultanti in liste (opzionale, ma utile per manipolazioni successive)
train_set = list(train_set)
train_labels = list(train_labels)

In [ ]:
print("Train set length: " + str(len(train_set)) + " Train label length: " + str(len(train_labels)))
print("Test set length: " + str(len(test_set)) + " Train label length: " + str(len(test_labels)))

In [ ]:
def show_images(inputs, labels=None, num_img=8, randomize=True):
    """
    Mostra un insieme di immagini da una lista di percorsi o array NumPy.

    Input:
        inputs: Lista di percorsi di immagini (str) o immagini come array NumPy (np.ndarray).
        labels: Lista di etichette corrispondenti alle immagini (opzionale).
        num_img: Numero di immagini da visualizzare (default: 8).
        randomize: Se True, seleziona le immagini casualmente (default: True).
    """
    # Se labels è None, crea una lista vuota di etichette
    if labels is None:
        labels = [None] * len(inputs)

    # Combina inputs e labels in una lista di tuple
    inputs_and_labels = list(zip(inputs, labels))

    # Seleziona casualmente le immagini e le etichette corrispondenti
    if randomize:
        selected = random.sample(inputs_and_labels, min(num_img, len(inputs)))
    else:
        selected = inputs_and_labels[:min(num_img, len(inputs))]

    # Crea la figura
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.ravel()

    for i, (input_item, label) in enumerate(selected):
        # Determina se l'input è un percorso o un'immagine diretta
        if isinstance(input_item, str):
            # Carica l'immagine dal percorso
            img = mpimg.imread(input_item)
        elif isinstance(input_item, np.ndarray):
            # Usa direttamente l'immagine come array NumPy
            img = input_item
        else:
            print(f"Tipo di input non supportato: {type(input_item)}")
            continue

        # Mostra l'immagine
        axes[i].imshow(img)
        axes[i].axis('off')  # Nasconde gli assi
        if label is not None:
            axes[i].set_title(f"Label: {bool(label)}")  # Converte 0/1 in False/True

    # Nasconde eventuali subplot vuoti
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
show_images(train_set, train_labels)

In [ ]:
# Verifica se il file del modello esiste
model_path = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(model_path):
    raise FileNotFoundError(
        f"File del modello non trovato: {model_path}. "
        "Scarica il modello da http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    )

# Inizializza il rilevatore di volti e il predittore di landmark di dlib
detector_dlib = dlib.get_frontal_face_detector()
predictor_dlib = dlib.shape_predictor(model_path)

def extract_landmarks(lista_di_input):
    """
    Estrae i landmark facciali da una lista di percorsi di immagini o da immagini come array NumPy.

    Input:
        lista_di_input: Lista di percorsi di immagini (str) o immagini come array NumPy (np.ndarray).

    Output:
        landmarks_list: Lista di array NumPy contenenti i landmark per ogni immagine.
                       Se non viene rilevato un volto o si verifica un errore, viene restituito None.
    """
    landmarks_list = []

    for input_item in tqdm(lista_di_input, desc="Estrazione landmark"):
        try:
            # Determina se l'input è un percorso o un'immagine diretta
            if isinstance(input_item, str):
                # Carica l'immagine dal percorso
                image = cv2.imread(input_item)
                if image is None:
                    print(f"Immagine non valida: {input_item}")
                    landmarks_list.append(None)
                    continue
            elif isinstance(input_item, np.ndarray):
                # Usa direttamente l'immagine come array NumPy
                image = input_item
            else:
                print(f"Tipo di input non supportato: {type(input_item)}")
                landmarks_list.append(None)
                continue

            # Converti in scala di grigi
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Rileva i volti nell'immagine
            faces = detector_dlib(image)
            if len(faces) == 0:
                print(f"Nessun volto rilevato in: {input_item}")
                landmarks_list.append(None)
                continue

            # Prendi il volto più grande (se ci sono più volti)
            face = max(faces, key=lambda f: f.width() * f.height())

            # Estrai i landmark
            landmarks = predictor_dlib(gray, face)
            landmarks = np.array([[p.x, p.y] for p in landmarks.parts()], dtype=np.int32)

            # Aggiungi i landmark alla lista
            landmarks_list.append(landmarks)

        except Exception as e:
            print(f"Errore durante l'elaborazione di {input_item}: {str(e)}")
            landmarks_list.append(None)

    return landmarks_list

In [ ]:
def show_images_with_landmarks(images, landmarks, num_images_to_print=8, randomize=True):
    """
    Visualizza le immagini con i landmark disegnati sopra.

    Input:
        images: Lista di immagini (formato: array NumPy o percorsi di file).
        landmarks: Lista di landmark allineati (formato: array NumPy di forma (68, 2) o None).
        num_images_to_print: Numero di immagini da visualizzare (default: 8).
        randomize: Se True, randomizza la selezione delle immagini (default: True).
    """
    # Se randomize è True, seleziona casualmente le immagini da visualizzare
    if randomize:
        indices = random.sample(range(len(images)), min(num_images_to_print, len(images)))
    else:
        indices = range(min(num_images_to_print, len(images)))

    # Crea una figura con un layout di subplot
    num_rows = int(np.ceil(num_images_to_print / 4))
    fig, axes = plt.subplots(num_rows, 4, figsize=(15, num_rows * 4))
    axes = axes.flatten()  # Appiattisce l'array di assi per facilitare l'iterazione

    for i, idx in enumerate(indices):
        img = images[idx]
        landmark = landmarks[idx]

        # Mostra l'immagine
        if isinstance(img, str):  # Se l'immagine è un percorso di file, caricala
            img = plt.imread(img)
        axes[i].imshow(img)
        axes[i].axis('off')  # Nasconde gli assi

        # Disegna i landmark se presenti
        if landmark is not None:
            axes[i].scatter(landmark[:, 0], landmark[:, 1], c='red', s=10)  # Disegna i punti rossi

    # Nasconde eventuali subplot vuoti
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Inizializza il rilevatore MTCNN
detector_mtcnn = MTCNN()

def extract_face_data(image_paths):
    """
    Estrae i dati delle facce (landmark, bounding box, ecc.) da una lista di immagini.

    Input:
        image_paths: Lista di percorsi di immagini.

    Output:
        face_data_list: Lista di dizionari contenenti i dati delle facce.
                       Ogni dizionario contiene:
                       - 'box': Coordinate del bounding box (x, y, w, h).
                       - 'keypoints': Coordinate dei 5 landmark.
                       - 'confidence': Livello di confidenza del rilevamento.
    """
    face_data_list = []

    for img_path in image_paths:
        # Carica l'immagine
        image = cv2.imread(img_path)
        if image is None:
            print(f"Immagine non valida: {img_path}")
            face_data_list.append(None)  # Aggiungi None per mantenere l'allineamento
            continue

        # Rileva i volti e i landmark nell'immagine
        results = detector_mtcnn.detect_faces(image, threshold_onet=0.5)
        if len(results) == 0:
            print(f"Nessun volto rilevato in: {img_path}")
            face_data_list.append(None)  # Aggiungi None per mantenere l'allineamento
            continue

        # Prendi il volto con la massima confidenza
        face = max(results, key=lambda f: f['confidence'])

        # Aggiungi i dati del volto alla lista
        face_data_list.append(face)

    return face_data_list

In [ ]:
import cv2

def crop_faces(image_paths, face_data_list, target_size=(224, 224), padding=50):
    """
    Ritaglia le facce dalle immagini originali utilizzando i dati delle facce, aggiungendo un padding.

    Input:
        image_paths: Lista di percorsi di immagini.
        face_data_list: Lista di dizionari contenenti i dati delle facce.
        target_size: Dimensione desiderata per le immagini ritagliate (default: 224x224).
        padding: Valore di padding da aggiungere al bounding box (default: 10).

    Output:
        cropped_faces: Lista di immagini ritagliate e ridimensionate.
    """
    cropped_faces = []

    for img_path, face_data in zip(image_paths, face_data_list):
        if face_data is None:
            print(f"Nessun dato del volto per: {img_path}")
            cropped_faces.append(None)
            continue

        # Carica l'immagine
        image = cv2.imread(img_path)
        if image is None:
            print(f"Immagine non valida: {img_path}")
            cropped_faces.append(None)
            continue

        # Ottieni le coordinate del bounding box
        x, y, w, h = face_data['box']

        # Applica il padding
        x = max(x - padding, 0)  # Evita coordinate negative
        y = max(y - padding, 0)  # Evita coordinate negative
        w = w + 2 * padding      # Aumenta la larghezza
        h = h + 2 * padding      # Aumenta l'altezza

        # Assicurati che le coordinate non superino i limiti dell'immagine
        x_end = min(x + w, image.shape[1])
        y_end = min(y + h, image.shape[0])
        w = x_end - x
        h = y_end - y

        # Ritaglia il volto
        face_image = image[y:y+h, x:x+w]

        # Ridimensiona l'immagine alla dimensione target
        face_image = cv2.resize(face_image, target_size)

        # Converti l'immagine in RGB (se necessario)
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)

        # Aggiungi l'immagine ritagliata alla lista
        cropped_faces.append(face_image)

    return cropped_faces

In [ ]:
train_set_short = train_set[:20]
train_labels_short = train_labels[:20]

In [ ]:
#face_data_list = extract_face_data(train_set_short)
#cropped_faces = crop_faces(train_set_short, face_data_list)

In [ ]:
#show_images(cropped_faces, train_labels_short)

In [ ]:
landmarks = extract_landmarks(train_set_short)

In [ ]:
show_images_with_landmarks(train_set_short, landmarks)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        """
        Args:
            image_paths (list): Lista di percorsi alle immagini.
            labels (list): Lista di etichette.
            transform (callable, optional): Trasformazioni da applicare alle immagini.
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Carica l'immagine dal percorso
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")  # Converti in RGB per sicurezza
        label = self.labels[idx]

        # Applica le trasformazioni se sono definite
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
torch.cuda.is_available()

In [ ]:
# Parametri
INPUT_SHAPE = (224, 224)  # Dimensione input per ResNet50
NUM_CLASSES = 1  # Classificazione binaria (REAL o FAKE)
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0001

# Percorso del dataset (sostituisci con i tuoi dati)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona l'immagine
    transforms.RandomHorizontalFlip(),  # Flip casuale
    transforms.ToTensor(),  # Converti in tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizza
])

# Crea il dataset personalizzato
train_dataset = CustomDataset(train_set, train_labels, transform=transform)
test_dataset = CustomDataset(test_set, test_labels, transform=transform)
# Crea il DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Carica ResNet50 pre-addestrato (senza i layer fully connected)
base_model = models.resnet50(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False  # Blocca i pesi di ResNet50 per il transfer learning iniziale

# Aggiungi layer personalizzati per la classificazione
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, NUM_CLASSES),
    nn.Sigmoid()
)

# Modello
model = base_model
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Funzione di perdita e ottimizzatore
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Addestramento iniziale (solo i layer personalizzati)
print("Addestramento della testa del modello...")
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.float().to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{EPOCHS}], Loss: {running_loss / len(train_loader):.4f}")

# Fine-tuning: sblocca alcuni layer di ResNet50 e riaddestra
print("Fine-tuning del modello...")
for param in base_model.parameters():
    param.requires_grad = True  # Sblocca tutti i layer

# Ottimizzatore con un tasso di apprendimento più basso
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE / 10)

# Addestramento con fine-tuning
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.float().to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{EPOCHS}], Loss: {running_loss / len(train_loader):.4f}")

# Salva il modello
torch.save(model.state_dict(), "resnet50_deepfake_model.pth")
print("Modello salvato con successo!")